In [11]:
import json
import re

def extract_qa_pairs(message):
    # Regex tìm các đoạn chứa Câu hỏi và Trả lời
    pattern = r"Câu hỏi:\n(.*?)\n===\nTrả lời:\n(.*?)(?=\n===\nCâu hỏi:|\Z)"
    matches = re.findall(pattern, message, re.DOTALL)
    
    # Xử lý danh sách kết quả để loại bỏ khoảng trắng thừa
    qa_pairs = [(q.strip(), a.strip()) for q, a in matches]
    
    return qa_pairs


In [12]:
with open("sample_check.json", "r") as f:
    data = json.load(f)

In [13]:
data

{'messages': 'Câu hỏi:\nBức ảnh này chụp bộ phận nào?\n===\nTrả lời:\nBức ảnh này chụp toàn bộ cơ thể của bệnh nhân.\n===\nCâu hỏi:\nCó phát hiện bất thường nào ở hệ thống xương không?\n===\nTrả lời:\nCó, hình ảnh cho thấy một nốt đặc xương đường kính 11 mm ở chỏm xương đùi trái. Tuy nhiên, nốt này không tăng chuyển hóa FDG, điều này gợi ý rằng khả năng cao là tổn thương lành tính.\n===\nCâu hỏi:\nCó tổn thương nào nghi ngờ ác tính ở hệ thống cơ xương không?\n===\nTrả lời:\nKhông, hình ảnh không ghi nhận bất thường nào có tăng chuyển hóa FDG trong hệ thống cơ xương. Điều này có nghĩa là chưa có dấu hiệu gợi ý tổn thương ác tính hoặc di căn xương trên phim chụp.\n===\nCâu hỏi:\nNhìn chung, có dấu hiệu nào gợi ý tổn thương ác tính trong hệ thống cơ xương không?\n===\nTrả lời:\nDựa trên hình ảnh, không có dấu hiệu rõ ràng của tổn thương á'}

In [14]:
qa_pairs = extract_qa_pairs(data["messages"])

# Print results
for i, (q, a) in enumerate(qa_pairs, 1):
    print(f"Q{i}: {q}")
    print(f"A{i}: {a}")
    print("-" * 50)


Q1: Bức ảnh này chụp bộ phận nào?
A1: Bức ảnh này chụp toàn bộ cơ thể của bệnh nhân.
--------------------------------------------------
Q2: Có phát hiện bất thường nào ở hệ thống xương không?
A2: Có, hình ảnh cho thấy một nốt đặc xương đường kính 11 mm ở chỏm xương đùi trái. Tuy nhiên, nốt này không tăng chuyển hóa FDG, điều này gợi ý rằng khả năng cao là tổn thương lành tính.
--------------------------------------------------
Q3: Có tổn thương nào nghi ngờ ác tính ở hệ thống cơ xương không?
A3: Không, hình ảnh không ghi nhận bất thường nào có tăng chuyển hóa FDG trong hệ thống cơ xương. Điều này có nghĩa là chưa có dấu hiệu gợi ý tổn thương ác tính hoặc di căn xương trên phim chụp.
--------------------------------------------------
Q4: Nhìn chung, có dấu hiệu nào gợi ý tổn thương ác tính trong hệ thống cơ xương không?
A4: Dựa trên hình ảnh, không có dấu hiệu rõ ràng của tổn thương á
--------------------------------------------------
